In [1]:
import os
os.makedirs("Resultados", exist_ok=True)
from google.colab import drive
drive.mount('/content/drive')
!pip install tsai
!pip install hyperopt

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.3/324.3 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.0/278.0 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 97.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 68.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━

In [ ]:
# ===============================================
# IMPORTAR BIBLIOTECAS
# ===============================================
import warnings
warnings.filterwarnings('ignore')

import os
import gc
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import torch
import time

from tsai.all import *
from fastai.callback.tracker import EarlyStoppingCallback
from sklearn.metrics import mean_squared_error, mean_absolute_error
from hyperopt import Trials, STATUS_OK, STATUS_FAIL, tpe, fmin, hp, space_eval
import statsmodels.api as sm
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

# ===============================================
# MONTAR O GOOGLE DRIVE
# ===============================================
from google.colab import drive
drive.mount('/content/drive')

# ===============================================
# DEFINIR PASTA PARA SALVAMENTO DENTRO DE "Colab Notebooks"
# ===============================================
colab_notebooks_path = "/content/drive/MyDrive/Colab Notebooks"
results_path = f"{colab_notebooks_path}/Resultados"

# Criar pasta "Resultados" dentro de "Colab Notebooks", se não existir
if not os.path.exists(results_path):
    os.makedirs(results_path)

# ===============================================
# CONFIGURAR GPU PARA TREINAMENTO RÁPIDO
# ===============================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Rodando em:", device)

# ===============================================
# CARREGAR O DATASET DO GOOGLE DRIVE
# ===============================================
file_path = "/content/drive/MyDrive/datasetBitcoin.csv"
df = pd.read_csv(file_path, parse_dates=['Date'], dayfirst=True, index_col='Date')

# Converter a coluna 'Price' para float (caso tenha separador de milhares)
df['Price'] = df['Price'].astype(str).str.replace(',', '').astype(float)

# Ordenar os dados por data
df = df.sort_index()

# ===============================================
# DEFINIR PARÂMETROS DO MODELO
# ===============================================
history = 24
horizon = 1
test_ratio = 0.2
max_evals = 50

model_name = 'OmniScaleCNN_Bitcoin'
input_features = ['Price']
data = df[input_features].values

# ===============================================
# SEPARAR DADOS EM TREINO E TESTE
# ===============================================
train_ind = int(len(df) * 0.8)
train = df[:train_ind]
test = df[train_ind:]

print('Training size:', train.shape[0])
print('Test size:', test.shape[0])

# ===============================================
# PREPARAR OS DADOS PARA A REDE NEURAL
# ===============================================
x_data, y_data = [], []
length = data.shape[0]

for i in range(length - history - horizon + 1):
    x = data[i:i+history, :]
    y = data[i+history:i+history+horizon, 0]
    x_data.append(x)
    y_data.append(y)

x_data = np.array(x_data)
y_data = np.array(y_data)

x_data = np.swapaxes(x_data, 1, 2)

test_length = len(test) - horizon + 1
train_valid_length = x_data.shape[0] - test_length
train_length = int(train_valid_length * 0.8)
valid_length = train_valid_length - train_length

X_train = x_data[:train_length]
y_train = y_data[:train_length]
X_valid = x_data[train_length:train_valid_length]
y_valid = y_data[train_length:train_valid_length]
X_test = x_data[train_valid_length:]
y_test = y_data[train_valid_length:]

# ===============================================
# CONFIGURAÇÃO DOS DADOS PARA O MODELO
# ===============================================
X, y, splits = combine_split_data([X_train, X_valid], [y_train, y_valid])
tfms = [None, [TSRegression()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits, inplace=True)

# ===============================================
# OTIMIZAÇÃO DOS HIPERPARÂMETROS
# ===============================================
search_space = {
    'batch_size': hp.choice('bs', [16, 32, 64, 128]),
    "lr": hp.choice('lr', [0.01, 0.001, 0.0001]),
    "epochs": hp.choice('epochs', [20, 50, 100]),
    "patience": hp.choice('patience', [5, 10]),
    "optimizer": hp.choice('optimizer', [Adam])
}

# ===============================================
# TREINAMENTO E OTIMIZAÇÃO
# ===============================================
def create_model_hypopt(params):
    try:
        gc.collect()
        batch_size = params["batch_size"]
        dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[batch_size, batch_size], num_workers=0)

        model = create_model(OmniScaleCNN, d=False, dls=dls).to(device)
        learn = Learner(dls, model, metrics=[mae, rmse], opt_func=params['optimizer'])
        learn.fit_one_cycle(params['epochs'], lr_max=params['lr'],
                            cbs=EarlyStoppingCallback(monitor='valid_loss', patience=params['patience']))
        val_loss = learn.recorder.values[-1][1]
        return {'loss': val_loss, 'status': STATUS_OK}

    except:
        return {'loss': None, 'status': STATUS_FAIL}

trials = Trials()
best = fmin(create_model_hypopt, space=search_space, algo=tpe.suggest, max_evals=max_evals, trials=trials)
params = space_eval(search_space, best)

# ===============================================
# TREINAMENTO FINAL DO MODELO
# ===============================================
batch_size = params["batch_size"]
dls = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[batch_size, batch_size], num_workers=0)
model = create_model(OmniScaleCNN, d=False, dls=dls).to(device)

learn = Learner(dls, model, metrics=[mae, rmse], opt_func=params['optimizer'])
learn.fit_one_cycle(params['epochs'], lr_max=params['lr'],
                    cbs=EarlyStoppingCallback(monitor='valid_loss', patience=params['patience']))

# Salvar gráfico de métricas dentro da pasta "Colab Notebooks"
learn.plot_metrics()
plt.savefig(f"{results_path}/{model_name}_metrics.pdf", bbox_inches='tight', pad_inches=0.1)

# ===============================================
# TESTE DO MODELO E AVALIAÇÃO FINAL
# ===============================================
test_ds = dls.valid.dataset.add_test(X_test, y_test)
test_dl = dls.valid.new(test_ds)
test_preds = learn.get_preds(dl=test_dl, with_decoded=True)[2]

y_pred = test_preds.numpy().squeeze()
y_true = y_test.squeeze()

# Salvar métricas
rmse = np.sqrt(mean_squared_error(y_true, y_pred))
mae = mean_absolute_error(y_true, y_pred)

print("RMSE:", rmse)
print("MAE:", mae)

# Salvar previsões e métricas no Google Drive dentro da pasta "Colab Notebooks"
pickle.dump(y_pred, open(f"{results_path}/OmniScaleCNN_Bitcoin_Pred.pkl", 'wb'))
pickle.dump(y_true, open(f"{results_path}/OmniScaleCNN_Bitcoin_True.pkl", 'wb'))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Rodando em: cpu
Training size: 4011
Test size: 1003
  0%|          | 0/50 [00:00<?, ?trial/s, best loss=?]

epoch,train_loss,valid_loss,mae,_rmse,time
0,395830592.000000,278150912.000000,9213.174805,16677.857422,00:10
1,401602816.000000,278142144.000000,9212.857422,16677.593750,00:17
2,408515104.000000,278130688.000000,9212.450195,16677.250000,00:10
3,402331296.000000,278117216.000000,9211.981445,16676.847656,00:10
4,400451168.000000,278104832.000000,9211.630859,16676.474609,00:08
5,405101280.000000,278077376.000000,9210.848633,16675.652344,00:10
6,407683392.000000,278058400.000000,9210.756836,16675.083984,00:12
7,409799744.000000,278006720.000000,9209.457031,16673.533203,00:10
8,408988960.000000,277970336.000000,9208.653320,16672.443359,00:08
9,410723424.000000,277901920.000000,9206.670898,16670.390625,00:10


No improvement since epoch 35: early stopping
  2%|▏         | 1/50 [07:38<6:14:13, 458.24s/trial, best loss: 276531200.0]

epoch,train_loss,valid_loss,mae,_rmse,time
0,410905056.000000,278108160.000000,9211.584961,16676.574219,00:09
1,406450528.000000,278015840.000000,9209.446289,16673.806641,00:09
2,407659648.000000,277768800.000000,9203.379883,16666.396484,00:09
3,403072000.000000,276707040.000000,9176.522461,16634.513672,00:08
4,403443840.000000,275804256.000000,9156.591797,16607.355469,00:09
5,398023680.000000,275014624.000000,9139.925781,16583.564453,00:09
6,398311968.000000,273274080.000000,9100.312500,16531.001953,00:08
7,395574976.000000,273970176.000000,9120.093750,16552.044922,00:09
8,399034592.000000,270065120.000000,9017.474609,16433.658203,00:09
9,401331168.000000,269152928.000000,9004.713867,16405.880859,00:09


  4%|▍         | 2/50 [10:49<4:01:08, 301.44s/trial, best loss: 265344384.0]

...

epoch,train_loss,valid_loss,mae,_rmse,time
0,404248576.000000,277856576.000000,9204.434570,16669.031250,00:09
1,406304320.000000,275383328.000000,9147.271484,16594.677734,00:09
2,402360352.000000,266612800.000000,8936.962891,16328.282227,00:08
3,386270848.000000,239270944.000000,8399.352539,15468.384766,00:09
4,350257984.000000,200897696.000000,7728.277344,14173.837891,00:09
5,296043296.000000,159677792.000000,6764.656738,12636.368164,00:08
6,237460176.000000,111981464.000000,6486.974121,10582.129883,00:09
7,182262240.000000,87073464.000000,6454.939453,9331.316406,00:09
8,144128368.000000,88743040.000000,5979.466309,9420.352539,00:08
9,117392088.000000,84220608.000000,7665.097168,9177.177734,00:09


 28%|██▊       | 14/50 [1:40:54<3:39:55, 366.54s/trial, best loss: 10591852.0]

epoch,train_loss,valid_loss,mae,_rmse,time
0,395772096.000000,278127264.000000,9212.279297,16677.148438,00:09
1,400774560.000000,278089024.000000,9210.848633,16676.001953,00:09
2,403797216.000000,278038912.000000,9209.823242,16674.498047,00:09
3,405728320.000000,277850656.000000,9204.861328,16668.853516,00:09
4,402341696.000000,277734208.000000,9202.231445,16665.359375,00:09
5,400697376.000000,277401088.000000,9194.041016,16655.363281,00:08
6,403048192.000000,276999968.000000,9184.787109,16643.316406,00:09
7,406760832.000000,276364128.000000,9170.401367,16624.203125,00:09
8,402062304.000000,275338432.000000,9146.671875,16593.326172,00:08
9,399439392.000000,274071680.000000,9114.410156,16555.109375,00:09


 30%|███       | 15/50 [1:48:42<3:51:40, 397.15s/trial, best loss: 10591852.0]

epoch,train_loss,valid_loss,mae,_rmse,time
0,412527712.000000,278147232.000000,9213.058594,16677.746094,00:10
1,387853824.000000,278137856.000000,9212.715820,16677.464844,00:09
2,402568672.000000,278128224.000000,9212.368164,16677.175781,00:10
3,402787616.000000,278118048.000000,9212.027344,16676.873047,00:10
4,406376864.000000,278104384.000000,9211.586914,16676.462891,00:10
5,402156512.000000,278087584.000000,9211.132812,16675.958984,00:09
6,409532160.000000,278069952.000000,9210.787109,16675.427734,00:10
7,406252352.000000,278042304.000000,9210.200195,16674.599609,00:10
8,403430624.000000,278001216.000000,9209.096680,16673.367188,00:10
9,409949216.000000,277969056.000000,9208.350586,16672.402344,00:10
